<a href="https://colab.research.google.com/github/Majethia/AI-Book-Summarizer/blob/main/Book_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai==0.28.1
!pip install tiktoken
!pip install chromadb
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followin

In [2]:
import getpass
import os

from langchain_community.document_loaders import BSHTMLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from google.colab import drive

drive.mount('/content/drive')


OPENAI_API_KEY = 'xyz'

Mounted at /content/drive


In [5]:
import requests
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

r = requests.get("https://gutenberg.org/cache/epub/84/pg84-images.html")
soup = BeautifulSoup(r.text)
b = soup.get_text()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([b])
print(len(texts))
print(texts[0])

db = Chroma.from_documents(texts, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY), persist_directory='/content/drive/MyDrive/Testing/book.db')

645
page_content='The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft Shelley\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\nTitle: Frankenstein; Or, The Modern Prometheus\n\nAuthor: Mary Wollstonecraft Shelley\n\nRelease date: October 1, 1993 [eBook #84]\r\n                Most recently updated: December 2, 2022\nLanguage: English\nCredits: Judith Boss, Christy Phillips, Lynn Hanninen and David Meltzer. HTML version by Al Haines.\r\n        Fu

In [6]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import textwrap

template = """You are an AI Book expert and you need to answer questions asked by the Human
{context}

{chat_history}
Human: {human_input}
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    OpenAI(openai_api_key=OPENAI_API_KEY, temperature=0), chain_type="stuff", memory=memory, prompt=prompt
)

query = ''

while query != "Exit":
  query = input("Human: ")
  docs = db.similarity_search(query)
  text = chain.invoke({"input_documents": docs, "human_input": query}, return_only_outputs=True)['output_text'].replace("AI: ", "")
  wrapped_text = textwrap.wrap(text, width=175)
  print("AI: ", end='')
  for i in wrapped_text:
    print(i)

print('\n\n\n')
print(memory)

Human: what is the title of the book?
AI:  The title of the book is "Frankenstein" by Mary Shelley.
Human: Summarize the plot for me
AI: The book follows the story of Victor Frankenstein, a young scientist who creates a living being from dead body parts. The creature, rejected by its creator and society, seeks
revenge and causes chaos in Frankenstein's life. The novel explores themes of ambition, responsibility, and the consequences of playing God.
Human: what happens in the first chapter
AI: In the first chapter, the narrator, Robert Walton, is on a ship bound for the North Pole when they encounter a man, Victor Frankenstein, floating on a piece of ice.
Frankenstein tells Walton his story of creating a living being and the consequences that followed.
Human: how does the story conclude does it have a happy ending?
AI: The story concludes with the death of both Victor Frankenstein and his creation. It can be argued that it does not have a happy ending, as both characters suffer greatly
